Congressiona Information

<h1> City Council District </h1>

In [1]:
#pip install sodapy

In [2]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofnewyork.us,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("uvw5-9znb", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

results_df.tail()

,name,district,borough,political_party
46,Ari Kagan,47,Brooklyn,Republican
47,Inna Vernikov,48,Brooklyn,Republican
48,Kamillah Hanks,49,Staten Island,Democrat
49,David Carr,50,Staten Island,Republican
50,Joseph Borelli,51,Staten Island,Republican


<h1>Congressional District</h1>

In [3]:
from bs4 import BeautifulSoup as bs
import requests as r
import pandas as pd
import re

In [4]:
url = "https://www.govtrack.us/congress/members/NY#representatives"

page = r.get(url)
soup = bs(page.text, "html")

In [5]:
columns_names = ["Congrssional District", "Representive Name", "Party Affiliate"]

df = pd.DataFrame(columns=columns_names)

representatives = soup.find_all("div", id = "representatives")

In [6]:
rep_name_list = []
congressional_district = []
democrat_or_republican = []

for rep in representatives:
    # get the representative name
    name = rep.find_all("a", class_ = "plain")
    
    # get the congressional District
    cong_dis = rep.find_all("div", class_ = ["photo_flag flag_d", "photo_flag flag_r"])

    # get the person position
    dr = rep.find_all("div", class_ = "info")

    # loop to turn the html into a text value
    for x in name:
        value = x.text
        rep_name_list.append(value)
    
    for x in cong_dis:
        cd = x.text.split()[0]
        cd_num_only = re.sub(r"[a-z]", "", cd)
        congressional_district.append(cd_num_only)

    for x in dr:
        dem_or_rep = x.text.split()[0]
        democrat_or_republican.append(dem_or_rep)

df["Congrssional District"] = congressional_district
df["Representive Name"] = rep_name_list
df["Party Affiliate"]= democrat_or_republican

df.tail(5)

,Congrssional District,Representive Name,Party Affiliate
21,22,Brandon Williams,Republican
22,23,Nicholas Langworthy,Republican
23,24,Claudia Tenney,Republican
24,25,Joseph Morelle,Democrat
25,26,Timothy Kennedy,Democrat


<h1>State Assembly</h1>

In [7]:
df_sa = pd.DataFrame(columns = ["State Assembly Number", "State Assembly Name"])

url = "https://nyassembly.gov/mem/"

page = r.get(url)
soup = bs(page.text, "html")

section = soup.find("section", class_ = "mem-wrapper")

In [8]:
SA_name_list = []
SA_number_list = []

SA_name = section.find_all("h3", class_ = "mem-name")

for sa in SA_name:
    SA_string = sa.text.split()
    SA_number = int(SA_string[-1])
    SA_number_list.append(SA_number)

    SA_string_name = SA_string[:-2]
    joined_string = " ".join(SA_string_name)
    SA_name_list.append(joined_string)

df_sa["State Assembly Name"] = SA_name_list
df_sa["State Assembly Number"] = SA_number_list

df_sa2 = df_sa.sort_values(by = "State Assembly Number", ascending=True)
    

In [9]:
df_sa2.head()

,State Assembly Number,State Assembly Name
137,1,"Fred W. Thiele, Jr."
58,2,Jodi Giglio
39,3,Joe DeStefano
51,4,Ed Flood
127,5,Doug Smith


<h1>State Senatorial Number </h1>

In [15]:
df_ss = pd.DataFrame(columns = ["State Senatorial Number", "State Assembly Name", "Party Affiliation"])

url = "https://www.nysenate.gov/senators-committees"

page = r.get(url)

soup = bs(page.text, "html")

In [23]:
all_ss = soup.find("div", class_ = "views-element-container")

In [55]:
ss_name_list = []
ss_number_list = []
ss_party_affiliation = []

ss_name = all_ss.find_all("h4", class_ = "nys-senator--name")
ss_number = all_ss.find_all("span", class_ = "nys-senator--district")
for ss in ss_name:
    ss_name_list.append(ss.text)

for ss_n in ss_number:
    string_info = ss_n.text.split()
    ss_number_info = string_info[-2]
    ss_num_only = re.sub(r"[a-z]", "", ss_number_info)
    ss_number_list.append(ss_num_only)

    party_aff = string_info[:-2]
    joined_string_pa = " ".join(party_aff)
    ss_party_affiliation.append(joined_string_pa)
    

In [61]:
df_ss["State Senatorial Number"] = ss_number_list
df_ss["State Assembly Name"] = ss_name_list
df_ss["Party Affiliation"] = ss_party_affiliation

df_ss

,State Senatorial Number,State Assembly Name,Party Affiliation
0,15,Joseph P. Addabbo Jr.,(D)
1,43,Jacob Ashby,(R)
2,36,Jamaal T. Bailey,(D)
3,57,George M. Borrello,"(R, C)"
4,46,Neil D. Breslin,"(D, WF)"
...,...,...,...
57,6,Kevin Thomas,(D)
58,49,Mark Walczyk,"(R, C)"
59,52,Lea Webb,"(D, WF)"
60,38,Bill Weber,"(R, C)"


In [65]:
new_pa_list = []

for values in df_ss['Party Affiliation']:
    if "D" in values:
        new_pa_list.append("Democrat")
    elif "R" in values:
        new_pa_list.append("Republican")
    else:
        new_pa_list.append("Independent")

        
df_ss["Party Affiliation"] = new_pa_list

In [71]:
df_ss.head(5)

,State Senatorial Number,State Assembly Name,Party Affiliation
0,15,Joseph P. Addabbo Jr.,Democrat
1,43,Jacob Ashby,Republican
2,36,Jamaal T. Bailey,Democrat
3,57,George M. Borrello,Republican
4,46,Neil D. Breslin,Democrat
